In [67]:
import pandas as pd
import re
import nltk
import emoji
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [28]:
import tweepy

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAM8j0AEAAAAA0U89kyhF1RpEJd1tFb3SaL800wo%3D047zPSpIojMHSZ32V5X6gKtLooptdoZ1dONUfnmM4sbggDjufB"

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [29]:
keywords = ["stock market", "finance", "economy"]
Query = " OR ".join(keywords) + " -is:retweet"

all_tweets = []


response = client.search_recent_tweets(
    query=Query,
    max_results=20, 
    tweet_fields=["created_at", "author_id"]
)
if response.data:
    for tweet in response.data:
        all_tweets.append({
            "text": tweet.text, 
            "created_at": tweet.created_at
        })

In [41]:
for tweet in all_tweets:
    print(tweet)

{'text': "The future of sports finance is here Investors will soon be able to buy into their favorite teams' ownership stakes", 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': "RT @YourAnonNews: As the stock market continues it's tumble there are a few MAGA who are voicing that they'd like a refund.\nNo. Fuck that.…", 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': '@chauffeur243 Il finance seulement les journalistes et communicateurs pour sa visibilité \n🤣🤣🤣🤣', 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': 'RT @nirvana_io: The AI-Native Financial Paradigm Shift\n\nTraditional finance was designed for human decision-makers, but the future belongs…', 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': 'Game-changer in the world of finance is the emergence of yield farming, where investors are rewarde

In [38]:
tweets_list = all_tweets.copy()

In [69]:
pd.DataFrame(all_tweets)

,text,created_at
0,The future of sports finance is here Investors...,2025-03-19 11:07:11+00:00
1,RT @YourAnonNews: As the stock market continue...,2025-03-19 11:07:11+00:00
2,@chauffeur243 Il finance seulement les journal...,2025-03-19 11:07:11+00:00
3,RT @nirvana_io: The AI-Native Financial Paradi...,2025-03-19 11:07:11+00:00
4,Game-changer in the world of finance is the em...,2025-03-19 11:07:11+00:00
5,RT @TheBonkKong: 🦍🦧🔥Launch 1 hour after pre-sa...,2025-03-19 11:07:11+00:00
6,@Egmbsc Regarding #IntelligentFuture and the i...,2025-03-19 11:07:11+00:00
7,Mozambique’s textile industry recorded slight ...,2025-03-19 11:07:10+00:00
8,RT @nirvana_io: The AI-Native Financial Paradi...,2025-03-19 11:07:08+00:00
9,RT @LauraLoomer: JUST IN:\n\n⁦@JDVance⁩ will s...,2025-03-19 11:07:08+00:00


In [68]:
nlp = spacy.load("en_core_web_sm")

def preprocess_tweets(tweet):
    tweet = emoji.replace_emoji(tweet,replace="")
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE) 
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = tweet.lower()
    doc = nlp(tweet)
    words = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    return " ".join(words)

cleaned_tweets = [preprocess_tweets(tweet['text']) for tweet in all_tweets]

In [70]:
pd.DataFrame(cleaned_tweets)

,0
0,future sport finance investor soon able buy fa...
1,rt stock market continue tumble maga voice d l...
2,il finance seulement les journaliste et commun...
3,rt ainative financial paradigm shift tradition...
4,gamechanger world finance emergence yield farm...
5,rt launch hour presale buy
6,intelligentfuture impact smart protocol global...
7,mozambiques textile industry record slight gro...
8,rt ainative financial paradigm shift tradition...
9,rt serve rnc finance chairman time history republ
